# Basic Setting

In [293]:
import matplotlib.pyplot as plt

import random
import numpy as np
import pandas as pd

import sympy
from sympy import Matrix, init_printing

from scipy.sparse.linalg import svds,eigs

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances

from time import time
import warnings
warnings.filterwarnings(action='ignore')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ATK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [294]:
track = pd.read_csv('1160_global_track_spotify.csv')
track = pd.DataFrame(track)
# track = track.rename(columns={'Unnamed: 0':'index'})
# track_t = track.drop(['artist_name', 'track_name', 'album_name', ])
track.head()

,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,artist_popularity,track_popularity,artist_followers,danceability,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Harry's House,['pop'],94,92,21210921,167303,...,6,-5.338,0.0557,0.3420,0.001010,0.3110,0.662,173.930,167303,4
1,1,6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny,Me Porto Bonito,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,97,49472302,178567,...,1,-5.105,0.0817,0.0901,0.000027,0.0933,0.425,92.005,178567,4
2,2,3k3NWokhRRkEPhCzPmV8TW,Bad Bunny,Ojitos Lindos,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,97,49472302,258299,...,3,-5.745,0.0413,0.0800,0.000001,0.5280,0.268,79.928,258299,4
3,3,75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill (A Deal With God),Hounds Of Love,"['art pop', 'art rock', 'baroque pop', 'new wa...",76,89,1046980,298933,...,10,-13.123,0.0550,0.7200,0.003140,0.0604,0.197,108.375,298933,4
4,4,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,95,49472302,243717,...,5,-5.198,0.2530,0.0993,0.000291,0.1260,0.187,106.672,243717,4


# Feature Engineering 

In [295]:
import re
 
def cleanText(readData):
    text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData) # 쉼표(,) 뺌
 
    return text

In [296]:
genre = []
for i in track['artist_genre']:
    if i == '[]':
        i = 'NA'
        genre.append(i.strip()) #"'[]'"
    else:
        i = cleanText(i)
        genre.append(i.strip())
track['genre'] = genre

In [297]:
track['track_popularity'] = track['track_popularity'] / 100 

In [298]:
track.head()

,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,artist_popularity,track_popularity,artist_followers,danceability,...,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,0,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Harry's House,['pop'],94,0.92,21210921,167303,...,-5.338,0.0557,0.3420,0.001010,0.3110,0.662,173.930,167303,4,pop
1,1,6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny,Me Porto Bonito,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.97,49472302,178567,...,-5.105,0.0817,0.0901,0.000027,0.0933,0.425,92.005,178567,4,"latin, reggaeton, trap latino"
2,2,3k3NWokhRRkEPhCzPmV8TW,Bad Bunny,Ojitos Lindos,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.97,49472302,258299,...,-5.745,0.0413,0.0800,0.000001,0.5280,0.268,79.928,258299,4,"latin, reggaeton, trap latino"
3,3,75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill (A Deal With God),Hounds Of Love,"['art pop', 'art rock', 'baroque pop', 'new wa...",76,0.89,1046980,298933,...,-13.123,0.0550,0.7200,0.003140,0.0604,0.197,108.375,298933,4,"art pop, art rock, baroque pop, new wave pop, ..."
4,4,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.95,49472302,243717,...,-5.198,0.2530,0.0993,0.000291,0.1260,0.187,106.672,243717,4,"latin, reggaeton, trap latino"


## Collecting User info

In [104]:
# import random
# songs = list(track['track_name'].values)
# song = random.sample(songs, 5)

# total_dictionary = {}
# qs=[]
# qs.append("무슨 곡이 가장 좋아요?   1) {}  2) {}  3) {}  4) {}  5) {}".format(song[0],song[1],song[2],song[3],song[4]))
# qs.append("어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운")
# qs.append("어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림")
# qs.append("고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔")
# qs.append("끝")


# for q in qs:
#     question = q
#     if question == "끝":
#         break
#     else:
#         total_dictionary[question] = ""

# for i in total_dictionary:
#     print(i)
#     answer = input()
#     total_dictionary[i] = answer 

# a = list(total_dictionary.items())
# answer_1 = a[0][1]
# answer_2 = a[1][1]
# answer_3 = a[2][1]
# answer_4 = a[3][1]

무슨 곡이 가장 좋아요?   1) Very, Slowly  2) 10,000 Hours (with Justin Bieber)  3) SOFA  4) 1  5) Milky Way Love
10,000 Hours
어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운
어두운
어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림
빠름
고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔
신남


## Genre Recommeder

In [313]:
# def recommend_genre(data, music_name, top=100):

#     # TF-IDF
#     tfidf = TfidfVectorizer(ngram_range=(1,2))
#     tf_genre = tfidf.fit_transform(track.genre)
#     tf_genre.shape

#     # 코사인 유사도
#     ts_genre = cosine_similarity(tf_genre, tf_genre).argsort()[:,::-1]

#     #특정 장르 정보 뽑아오기
#     target_genre_index = data[data['track_name'] == music_name].index.values
    
#     #타겟 장르와 비슷한 코사인 유사도 값
#     sim_genre_index = ts_genre[target_genre_index, :top].reshape(-1)
#     #본인제외
#     sim_genre_index = sim_genre_index[sim_genre_index != target_genre_index]
    
# #     data["similarity_genre"] = ts_genre[target_genre_index, :top].reshape(-1,1)
    
#     #추천결과를 아티스트 팔로워 기준으로 정렬
# #     result_genre = data.iloc[sim_genre_index].sort_values(by='track_popularity', ascending=False)[:10]
#     result_genre = data.iloc[sim_genre_index][:10]
    
#     return result_genre[['artist_name', 'track_name', 'track_popularity']]

## Genre + Artist/Track/Album name Recommeder

In [300]:
# def recommend_genre(data, music_name, top=100):

#     # TF-IDF
#     tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2) ,stop_words=stopwords.words('english'))
#     tf_genre = tfidf.fit_transform(data['artist_name'] + ' ' +
#                                    data['track_name'] + ' ' +
#                                    data['album_name'] + ' ' +
#                                    data['genre'])

#     # 코사인 유사도
#     ts_genre = cosine_similarity(tf_genre, tf_genre).argsort()[:,::-1]

#     #특정 장르 정보 뽑아오기
#     target_genre_index = data[data['track_name'] == music_name].index.values
    
#     #타겟 장르와 비슷한 코사인 유사도 값
#     sim_genre_index = ts_genre[target_genre_index, :top].reshape(-1)
#     #본인제외
#     sim_genre_index = sim_genre_index[sim_genre_index != target_genre_index]
    
# #     data["similarity_genre"] = ts_genre[target_genre_index, :top].reshape(-1,1)
    
#     #추천결과를 아티스트 팔로워 기준으로 정렬
#     result_genre = data.iloc[sim_genre_index].sort_values(by='track_popularity', ascending=False)[:10]
# #     result_genre = data.iloc[sim_genre_index][:5]
    
#     return result_genre[['artist_name', 'track_name', 'track_popularity']]

# Content-Based Recommeder
- When a User first signs up on website

In [329]:
class ContentBasedRecommender:
    
    def __init__(self, data):
        self.data = data
        self.music = ''
        self.mood = ''
        self.speed = ''
        self.emotion = ''

        
    def user_info(self):
        print("--------------------------------------------------------------------\n\
노래를 추천해드리기 전에 잠시 당신에 대해서 알아보겠습니다 \n\
----------------------------------------------------------------------")
        songs = list(self.data['track_name'].values)
        song = random.sample(songs, 5)

        total_dictionary = {}
        qs=[]
        qs.append("무슨 곡이 가장 좋아요?   1) {}  2) {}  3) {}  4) {}  5) {}".format(song[0],song[1],song[2],song[3],song[4]))
        qs.append("어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운")
        qs.append("어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림")
        qs.append("고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔")
        qs.append("끝")

        for q in qs:
            question = q
            if question == "끝":
                break
            else:
                total_dictionary[question] = ""

        for i in total_dictionary:
            print(i)
            answer = input()
            total_dictionary[i] = answer 

        a = list(total_dictionary.items())
        self.music = a[0][1]
        self.mood = a[1][1]
        self.speed = a[2][1]
        self.emotion = a[3][1]
        print("오호 당신은 이런 사람이군요 ㅎㅅㅎ")
        return [self.music, self.mood, self.speed, self.emotion]
        
        
    def cosine_sim_data(self):    
        
        # TF-IDF
        tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2) ,stop_words=stopwords.words('english'))
        tf_genre = tfidf.fit_transform(self.data['artist_name'] + ' ' +
                                       self.data['track_name'] + ' ' +
                                       self.data['album_name'] + ' ' +
                                       self.data['genre'])

        # 코사인 유사도
        ts_genre = cosine_similarity(tf_genre, tf_genre).argsort()[:,::-1]
        return ts_genre

    
    def recommend_genre(self, music_name, cos_sim, top=100):
        
        print("--------------------------------------------------------------\n\
아티스트 / 트랙 / 앨범 / 장르에 따라 추천을 해드리겠습니다 \n\
---------------------------------------------------------------")
        
        #특정 장르 정보 뽑아오기
        target_genre_index = self.data[self.data['track_name'] == music_name].index.values

        #타겟 장르와 비슷한 코사인 유사도 값
        sim_genre_index = cos_sim[target_genre_index, :top].reshape(-1)
        #본인제외
        sim_genre_index = sim_genre_index[sim_genre_index != target_genre_index]
        
        #추천결과를 popularity로 기준으로 정렬
        result_genre = self.data.iloc[sim_genre_index].sort_values(by='track_popularity', ascending=False)[:10]
    #     result_genre = data.iloc[sim_genre_index][:5]

        return result_genre[['artist_name', 'track_name', 'track_popularity']]

    

a = ContentBasedRecommender(track)
b = a.user_info()
c = a.cosine_sim_data()
d = a.recommend_genre(b[0], c)
d

--------------------------------------------------------------------
노래를 추천해드리기 전에 잠시 당신에 대해서 알아보겠습니다 
----------------------------------------------------------------------
무슨 곡이 가장 좋아요?   1) A midsummer night's sweetness  2) 그런일은  3) 발자국  4) U MAD  5) Come Back Home
Come Back Home
어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운
어두운
어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림
빠름
고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔
평범
오호 당신은 이런 사람이군요 ㅎㅅㅎ
--------------------------------------------------------------
아티스트 / 트랙 / 앨범 / 장르에 따라 추천을 해드리겠습니다 
---------------------------------------------------------------


,artist_name,track_name,track_popularity
8,Lizzo,About Damn Time,0.94
34,Kendrick Lamar,N95,0.90
44,Future,WAIT FOR U (feat. Drake & Tems),0.89
14,The Kid LAROI,STAY (with Justin Bieber),0.87
31,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),0.85
11,Jack Harlow,First Class,0.84
105,PSY,Gangnam Style (강남스타일),0.73
664,Brent Faiyaz,MERCEDES,0.68
645,Syd,Out Loud (feat. Kehlani),0.65
704,Bryson Tiller,7:00,0.60


# Content-Based Recommender 
- When website does not have enough users

# Collaborative Filtering Recommender (User들이 어느정도 모였을때)
- When there are enough users